In [0]:
import re
import bs4
from urllib.request import urlopen

In [2]:
url_float = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=005930'
# Floating ratio from Naver
url_float

'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=005930'

In [0]:
source = urlopen(url_float).read()
soup = bs4.BeautifulSoup(source, 'lxml')

In [4]:
soup.find(id = 'cTB11').find_all('tr')[6].td.text

'\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t5,969,782,550주 / 74.48%\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t'

In [5]:
tmp = soup.find(id = 'cTB11').find_all('tr')[6].td.text
tmp = tmp.replace('\r', '')
tmp = tmp.replace('\n', '')
tmp = tmp.replace('\t', '')
tmp

'5,969,782,550주 / 74.48%'

In [6]:
tmp = re.split('/', tmp)
tmp

['5,969,782,550주 ', ' 74.48%']

In [7]:
tmp[0]

'5,969,782,550주 '

In [8]:
tmp[1]

' 74.48%'

In [9]:
outstanding = tmp[0].replace(',', '')
outstanding = outstanding.replace('주', '')
outstanding = outstanding.replace(' ', '')
outstanding

'5969782550'

In [10]:
floating = tmp[1].replace(' ', '')
floating = floating.replace('%', '')
floating

'74.48'

In [11]:
outstanding = int(outstanding)
outstanding

5969782550

In [0]:
# 구성종목 기본정보 
def stock_info(stock_cd):
  url_float = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + stock_cd
  source = urlopen(url_float).read()
  soup = bs4.BeautifulSoup(source, 'lxml')
  
  tmp = soup.find(id = 'cTB11').find_all('tr')[6].td.text
  tmp = tmp.replace('\r', '')
  tmp = tmp.replace('\n', '')
  tmp = tmp.replace('\t', '')
  
  tmp = re.split('/', tmp)
  
  outstanding = tmp[0].replace(',', '')
  outstanding = outstanding.replace('주', '')
  outstanding = outstanding.replace(' ', '')
  outstanding = int(outstanding)
  
  floating = tmp[1].replace(' ', '')
  floating = floating.replace('%', '')
  floating = float(floating)
  
  name = soup.find(id = 'pArea').find('div').find('div').find('tr').find('td').find('span').text
  
  k10_outstanding[stock_cd] = outstanding 
  k10_floating[stock_cd] = floating
  k10_name[stock_cd] = name

In [0]:
"""
한국거래소 시가총액 상위 10종목 (2019년 9월 2일 기준)
005930 삼성전자
000660 SK하이닉스
005935 삼성전자우
005380 현대차
035420 NAVER
012330 현대모비스
051910 LG화학
068270 셀트리온
055550 신한지주
017670 SK텔레콤
"""

k10_component = ['005930', '000660', '005935', '005380', '035420', '012330', '051910', '068270', '055550', '017670']

In [0]:
k10_outstanding = dict() # 딕셔너리 초기화
k10_floating = dict()
k10_name = dict()
for stock_cd in k10_component: # 종목을 하나씩 순환하면서
  stock_info(stock_cd)

In [15]:
k10_outstanding

{'000660': 728002365,
 '005380': 213668187,
 '005930': 5969782550,
 '005935': 5969782550,
 '012330': 95306694,
 '017670': 80745711,
 '035420': 164813395,
 '051910': 70592343,
 '055550': 474199587,
 '068270': 128336328}

In [16]:
k10_floating # 유동 비율

{'000660': 73.53,
 '005380': 65.62,
 '005930': 74.48,
 '005935': 74.48,
 '012330': 66.41,
 '017670': 52.57,
 '035420': 77.72,
 '051910': 64.28,
 '055550': 80.02,
 '068270': 75.83}

In [0]:
import datetime as dt
import pandas as pd

In [0]:
def date_format(d):
    d = str(d).replace('-', '.')
    
    yyyy = int(d.split('.')[0]) 
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])

    this_date= dt.date(yyyy, mm, dd)
    return this_date

In [0]:
def historical_stock_naver(stock_cd, start_date = '', end_date = '', page_n = 1, last_page = 0):
  
  if start_date: # start_date가 있으면
    start_date = date_format(start_date) # date 포맷으로 변환
  else:  # 없으면
    start_date = dt.date.today() # 오늘 날짜를 지정
  if end_date: # end_date가 있으면 
    end_date = date_format(end_date)
  else:
    end_date = dt.date.today()
  
  naver_stock = 'https://finance.naver.com/item/sise_day.nhn?code=' + stock_cd + '&page=' + str(page_n)
  
  source = urlopen(naver_stock).read()
  source = bs4.BeautifulSoup(source, 'lxml')
  
  dates = source.find_all('span', class_ = 'tah p10 gray03') # 날짜 수집
  prices = source.find_all('td', class_ = 'num') # 종가 수집
  
  for n in range(len(dates)):
    if len(dates) > 0 :
      
      # 날짜 처리
      this_date = dates[n].text
      this_date = date_format(this_date)
      
      if this_date <= end_date and this_date >= start_date:
        # start_date와 end_date 사이에서 데이터 저장
        # 종가 처리
        this_close = prices[n*6].text
        this_close = this_close.replace(',', '')
        this_close = float(this_close)
        
        # 딕셔너리에 저장
        historical_prices[this_date] = this_close
        
      elif this_date < start_date:
        # start_date 이전이면 함수 종료
        return historical_prices
  # 페이지 내비게이션
  if last_page == 0:
    last_page = source.find_all('table')[1].find('td', class_ = 'pgRR').find('a')['href']
    last_page = last_page.split('&')[1]
    last_page = last_page.split('=')[1]
    last_page = float(last_page)
    
  # 다음 페이지 호출
  if page_n < last_page:
    page_n = page_n + 1
    historical_stock_naver(stock_cd, start_date, end_date, page_n, last_page)
    
  return historical_prices

In [0]:
k10_historical_prices = dict()

for stock_cd in k10_component:
  
  historical_prices = dict()
  start_date = '2017-1-1'
  end_date = '2017-12-31'
  historical_stock_naver(stock_cd, start_date, end_date)
  
  k10_historical_prices[stock_cd] = historical_prices

In [0]:
k10_historical_price = pd.DataFrame(k10_historical_prices)

In [0]:
k10_historical_price = k10_historical_price.fillna(method = 'ffill') # ffill로 구멍을 채우고
if k10_historical_price.isnull().values.any() # 그래도 구멍이 남아 있으면
  k10_historical_price = k10_historical_price.fillna(method = 'bfill') # bfill로 채우기
k10_historical_price.head(3)

In [0]:
k10_historical_price['005930'] = k10_historical_price['005930'] / 50 # 삼성전자 액면분할에 따른 수정주가 계산
k10_historical_price.head(3)

In [0]:
tmp = {'Outstanding' : k10_outstanding, 'Floating': k10_floating, 'Price' : k10_historical_price.iloc[0], 'Name' : k10_name}
k10_info = pd.DataFrame(tmp)

In [0]:
k10_info['f Market Cap'] = k10_info['Outstanding'] * k10_info['Floating'] * k10_info['Price'] * 0.01
k10_info['Market Cap'] = k10_info['Outstanding'] * k10_info['Price'] * 0.01
k10_info.head(3)

In [0]:
k10_historical_mc = k10_historical_price * k10_info['Outstanding'] * k10_info['Floating'] * 0.01
k10_historical_mc.head(3)

In [0]:
"""
<데이터프레임>.sum()은 각 열의 합(세로 방향)
<데이터프레임>.sum(axis = 1)은 각 행의 합(가로 방향)
"""
k10_historical_mc.sum(axis = 1) # 일자별 시가 총액 합


In [0]:
k10 = pd.DataFrame() # k10 데이터프레임 초기화
k10['K10 Market Cap'] = k10_historical_mc.sum(axis = 1)
# 일자별 시가총액 저장
k10.head(3)

In [0]:
k10['K10'] = k10['K10 Market Cap'] / k10['K10 Market Cap'][0] * 100
k10.head(3)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
plt.figure(figsize = (10, 5))
plt.plot(k10['K10'])
plt.legend(loc = 0)
plt.grid(True, color = '0.7', linestyle= ':', linewidth = 1)

In [0]:
def historical_index_naver(index_cd, page_n=1, last_page=0):   
        
    naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    
    source = urlopen(naver_index).read()   # 지정한 페이지에서 코드 읽기
    source = bs4.BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
    
    dates = source.find_all('td', class_='date')   # <td class="date">태그에서 날짜 수집   
    prices = source.find_all('td', class_='number_1')   # <td class="number_1">태그에서 지수 수집
    
    for n in range(len(dates)):
    
        if dates[n].text.split('.')[0].isdigit():
            
            # 날짜 처리
            this_date = dates[n].text
            this_date= date_format(this_date)
            
            # 종가 처리
            this_close = prices[n*4].text   # prices 중 종가지수인 0,4,8,...번째 데이터 추출
            this_close = this_close.replace(',', '')
            this_close = float(this_close)

            # 딕셔너리에 저장
            historical_prices[this_date] = this_close
            
    # 페이지 네비게이션
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        # 마지막페이지 주소 추출
        last_page = last_page.split('&')[1]   # & 뒤의 page=506 부분 추출
        last_page = last_page.split('=')[1]   # = 뒤의 페이지번호만 추출
        last_page = int(last_page)   # 숫자형 변수로 변환
        
    # 다음 페이지 호출
    if page_n < last_page:   
        page_n = page_n + 1   
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)   
        
    return historical_prices  

In [0]:
historical_prices = dict() # 딕셔너리 초기화
kospi200 = historical_index_naver('KPI200','2017-1-1', '2017-12-31')
kospi200

In [0]:
k200 = pd.DataFrame({'K200' : kospi200})
k200.head(3)

In [0]:
plt.figure(figsize = (10, 5))
plt.plot(k10['K10'] / k10['K10'][0] * 100)
plt.plot(k200['K200'] / k200['K200'][0] * 100)
plt.legend(loc = 0)
plt.grid(True, color = '0.7', linestyle = ':', linewidth = 1)